## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-24-08-55-28 +0000,bbc,"Kyiv can win all of Ukraine back from Russia, ...",https://www.bbc.com/news/articles/c07vm35rryeo...
1,2025-09-24-08-35-51 +0000,nyt,Typhoon Ragasa Kills 14 in Taiwan as Hong Kong...,https://www.nytimes.com/2025/09/24/world/asia/...
2,2025-09-24-08-34-24 +0000,bbc,Urgent review ordered into asylum seeker taxi ...,https://www.bbc.com/news/articles/cx2j3regpdno...
3,2025-09-24-08-32-22 +0000,nyt,"Jimmy Kimmel, Somber but Defiant, Defends Free...",https://www.nytimes.com/2025/09/23/business/me...
4,2025-09-24-08-32-00 +0000,wsj,German Companies No Longer Feeling Cheerier as...,https://www.wsj.com/economy/german-companies-n...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,trump,76
23,kimmel,22
22,jimmy,18
128,was,13
383,un,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
42,2025-09-24-04-01-42 +0000,nypost,"Defiant, emotional Jimmy Kimmel returns to the...",https://nypost.com/2025/09/24/us-news/defiant-...,155
68,2025-09-24-01-43-00 +0000,wsj,When Trump announced changes to the H-1B visa ...,https://www.wsj.com/politics/policy/the-franti...,151
32,2025-09-24-05-04-00 +0000,wsj,Kimmel Makes Emotional Return to ABC With Trum...,https://www.wsj.com/business/media/jimmy-kimme...,137
171,2025-09-23-19-35-00 +0000,wsj,President Trump attacked global immigration an...,https://www.wsj.com/politics/trump-says-migrat...,134
290,2025-09-23-12-21-27 +0000,nypost,Trump to tout US as defender of Western civili...,https://nypost.com/2025/09/23/us-news/trump-to...,124


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
42,155,2025-09-24-04-01-42 +0000,nypost,"Defiant, emotional Jimmy Kimmel returns to the...",https://nypost.com/2025/09/24/us-news/defiant-...
68,69,2025-09-24-01-43-00 +0000,wsj,When Trump announced changes to the H-1B visa ...,https://www.wsj.com/politics/policy/the-franti...
15,56,2025-09-24-07-17-49 +0000,bbc,Jimmy Kimmel calls out 'anti-American' threats...,https://www.bbc.com/news/articles/c9dx6dlelw8o...
21,51,2025-09-24-07-04-00 +0000,wsj,China said it will no longer seek new special ...,https://www.wsj.com/economy/trade/china-to-sto...
171,50,2025-09-23-19-35-00 +0000,wsj,President Trump attacked global immigration an...,https://www.wsj.com/politics/trump-says-migrat...
200,49,2025-09-23-17-53-00 +0000,wsj,Fed Chair Jerome Powell said he judged the Fed...,https://www.wsj.com/economy/central-banking/fe...
213,41,2025-09-23-17-29-24 +0000,nypost,Megyn Kelly slams ‘coward’ Van Jones for brand...,https://nypost.com/2025/09/23/media/megyn-kell...
275,38,2025-09-23-13-46-30 +0000,bbc,Secret Service disrupts telecom threat near UN...,https://www.bbc.com/news/articles/cn4w0d8zz22o...
56,35,2025-09-24-02-36-48 +0000,nypost,24-year-old illegal immigrant who posed as Ohi...,https://nypost.com/2025/09/23/us-news/24-year-...
35,31,2025-09-24-04-25-06 +0000,nypost,Camp Mystic plans to reopen with new safety pr...,https://nypost.com/2025/09/24/us-news/camp-mys...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
